# Vue私房手册

如果没有注明，则皆为Vue2版本

## Vue核心

### 基础

#### Vue.use()

- [关于Vue.use详解](https://www.jianshu.com/p/89a05706917a)

#### 花括号与django冲突怎么办？自定义分隔符

如果使用django作为后台，则django的`{{}}`与Vue的冲突，那么最简单的办法，是在创建新的Vue实例的时候，声明`delimiters`属性，如下：
```javascript
var app = new Vue({
    delimiters: ['{', '}'],
    el: '#app',
    data: { message: 'Hello Vue!' },
})
```
上面的例子中，表明使用`{}`单花括号引用Vue的属性。

#### v-for为什么要加key

- [VUE中演示v-for为什么要加key](https://www.jianshu.com/p/4bd5e745ce95)
- [深入浅出 Vue 中的 key 值](https://blog.csdn.net/z591102/article/details/105091810)
- [知乎：Vue2.0 v-for 中 :key 到底有什么用？](https://www.zhihu.com/question/61064119/answer/766607894)

#### v-model

- [使用Vue.js创建自定义输入](https://www.smashingmagazine.com/2017/08/creating-custom-inputs-vue-js/)


1. checkbox使用`v-model`时要注意，如果`v-model`绑定的是字符串，那么绑定到这个字符串的是checkbox的`checked`属性。如果`v-model`绑定的是一个数组，那么会根据`checked`的值从数组中添加或者删除`value`值。
2. 多个radio如果`v-model`绑定的是同一个变量，那么在几个radio就可以进行相互切换。
3. select绑定的变量会匹配option的value，如果没有定义value，则会匹配option内的值。另外如果变量的初始值匹配不上所有的option的值，select的初始值会是空，最好是设置一个不能选择的初始值。如果想要多选，设置select为multiple, 则绑定的变量要是一个数组。

##### 文本框

对于文本`input`，`type`为`text`,`email`,`password`类型或者标签为`textarea`:
```html
<div><input v-model="message" placeholder="edit me">
<p>Message: {{ message }}</p>

<!-- OR -->

<p>message:</p>
<p style="white-space: pre-line">{{ message }}</p>
<textarea v-model="message" placeholder="add multiple lines"></textarea>
</div>
```
`v-model`实际上等价于：
```html
<textarea v-model="varName"等价于:value="varName" @input="e => varName = e.target.value"></textarea>
```

##### 单选按钮

对于单选按钮，比如：
```html
<div>
  <input type="radio" value="One" v-model="picked">
  <input type="radio" value="Two" v-model="picked">
  <span>Picked: {{ picked }}</span>
</div>
```
v-model相当于下面语句的语法糖：
```html
<div>
  <input type="radio" value="One" :checked="picked == 'One'" @change="e => picked = e.target.value">
  <input type="radio" value="Two" :checked="picked == 'Two'" @change="e => picked = e.target.value">
  <span>Picked: {{ picked }}</span>
</div>
```

##### 复选框

复选框比较复杂，因为他有两种不同的行为，如果单个复选框，v-model则将其视为布尔值，并忽略value，如：
```html
<div>
  <input type="checkbox" value="foo" v-model="isChecked">
</div>
```
他相当于：
```html
<div>
  <input type="checkbox" value="foo" :checked="!!isChecked" @change="e => isChecked = e.target.checked">
</div>
```
如果希望它不是`true`和`false`，则可以使用`true-value`和`false-value`属性，该属性控制在选中或不选中此复选框时模型将设置为哪些值，比如：
```html
<div>
  <input type="checkbox" value="foo" v-model="isChecked" true-value="1" false-value="0">
</div>
```
相当于：
```html
<div>
  <input type="checkbox" value="foo" :checked="isChecked == '1'" @change="e => isChecked = e.target.checked ? '1' : '0'">
</div>
```
但如果有多个共享一个`v-model`的复选框，则这些复选框将使用所有已选中复选框的值填充一个数组。而且，`true-value`和`false-value`属性不再影响任何东西。比如：
```html
<div>
<template>
  <div>
    <input type="checkbox" value="foo" v-model="checkedVals">
    <input type="checkbox" value="bar" v-model="checkedVals">
    <input type="checkbox" value="baz" v-model="checkedVals">
  </div>
</template>
<script>
  export default {
    data: () => ({
      checkedVals: ['bar']
    })
  }
</script>
</div>
```
相当于：
```html
<div id="app">
  <div>
    <input type="checkbox" value="foo" :checked="shouldBeChecked('foo')" @change="updateVals">
    <input type="checkbox" value="bar" :checked="shouldBeChecked('bar')" @change="updateVals">
    <input type="checkbox" value="baz" :checked="shouldBeChecked('baz')" @change="updateVals">
  </div>
  <div>
    {{ checkedVals }}
  </div>
</div>
<script>
  new Vue({
    el: '#app',
    data: {
      checkedVals: ['bar']
    },
    methods: {
      shouldBeChecked(val) {
        return this.checkedVals.includes(val)
      },
      updateVals(e) {
        let isChecked = e.target.checked
        let val = e.target.value

        if (isChecked) {
          this.checkedVals.push(val)
        } else {
          this.checkedVals.splice(this.checkedVals.indexOf(val), 1)
        }
      }
    }
  })
</script>
```

<span style="color:red;font-weight:bold">注意，在Vue3中，v-model传递的value，即使不在组件的`props`里面声明，也不会包含在$attrs里面。</span>

#### style使用scope

- [vue中style下scope的使用和坑](https://www.cnblogs.com/makai/p/11415156.html)

#### template标签

`template`标签在普通的html中使用和在`vue`绑定的`el`中使用，有很大的区别。

- [Html5中的<template>标签](https://www.imooc.com/article/9415?block_id=tuijian_wz)
- [关于template标签用法总结(含vue中的用法总结)](https://blog.csdn.net/u010510187/article/details/100356624)

    
特别是在条件渲染中使用`template`，比如:
    
```html
<template v-if="show">
  show me
</template>
```
渲染出来就只有文本`show me`，而如果不使用`template`，如：
```html
<div v-if="show">
  show me
</div>
```
渲染出来会包含`div`元素。

#### 动态绑定事件

可以利用对象语法，通过`v-on="refreshByClick?{click:refreshCode}:{}"`这种方式动态绑定事件。

#### $watch注意事项

`$watch`有几个点要注意：
1. 如果第一个参数是函数，则会跟踪函数返回的结果，如果结果发生变化，则出发回调。比如：
```javascript
    this.$watch(
        () => this.$route.params,
        () => {
          this.loading = true
          this.blogContent = null
          setTimeout(() => {
            this.blogContent = "blog content"
            this.loading = false
          }, 1000)
        },
        {immediate: true}
    )
```
这是一段跟踪路由变化获取数据的代码，注意`this.$route.params`不能加大括号，当加大括号，返回会是`undefined`，此时不会触发回调。
2. immediate的作用，仍以上面的例子说明，第一次进入页面，此时是不会触发的，因为路由未发生变化，只是初始化。因此要加上immediate，表示立即触发。

#### 全局错误捕获

Vue3中，全局的`errorHandler`捕获不到`setTimeout`这样的异步错误，感觉vue的各个生命周期构成完整的一个异步链，在方法里面抛出的异常会顺着这个异步链传播，一直到最后的`errorHandler`，因此如果是在方法中孤立的promise中触发异常，是不会顺着外层的异步链传播下去的。只有return这个promise，异常才会合并到外层的promise链。  
所以，在then里面的promise，只有return以后才能继续在promise chain链上传播，否则就是一个孤立的promise,vue的全局errorHandler无法捕获，而全局的errorHandler感觉是在promise chain链的最底端。也可以使用await，await会强制返回一个promise.

在Vue3中如果await后面抛出了错误，会立刻抛出，同时await会返回一个`rejected promise`，最后又被全局的错误处理函数捕获。就像下面的代码：
```javascript
await (async function () {
  throw Error("error happened!")
})()
```
会立刻抛出错误，同时全局错误处理函数也会捕获这个错误，也就是说，就算设置了全局错误处理函数，`await`后面的错误一样会抛出，不会说`await`后面的错误就不抛出，由全局错误处理函数进行处理了。这个和普通的Promise链不同，普通的Promise链只要发生错误，都会由后面的catch捕获。注意，如果不await或者return，则全局的错误处理函数就无法捕获这个错误。

#### 计算属性

1. 计算属性里面如果要用到this，注意不能使用箭头函数，会导致this丢失的问题

### 组件

#### v-model如何在组件上工作

Vue并不知道组件具体是如何工作的，因此它使用统一的方式处理`v-model`，和文本的`input`相同，比如：
```html
<div>
  <my-custom-component v-model="myProperty" />
</div>
```
相当于:
```html
<div>
  <my-custom-component :value="myProperty" @input="val => myProperty = val" />
</div>
```
传入组件的是`value`属性，绑定的事件是`input`事件，可以通过组件的`model`属性来修改传入的属性和绑定的事件，比如在组件中：
```javascript
export default {
  name: 'my-custom-component',
  model: {
    prop: 'foo',
    event: 'bar'
  },
  // ...
}
```

##### 自定义radio

自定义的radio中，v-model流程是这样的：
1. 通过model将v-model绑定一个自定义的属性modelVal，将value和modelVal作为props传入组件，v-model的逻辑为:model-val=picked
2. 组件内部，radio的checked属性的值是modelVal==value的结果
3. 组件内部change事件向外发射同名事件，同时将value发射出去
4. 外部v-model的事件逻辑为picked=value
5. picked发生改变，下一个tick重新渲染组件

源码如下：
```html
  <body>
    <div id="app">
      <!-- 这里的v-model就相当于<my-radio :model-val="picked" @change="val => picked = val"></my-radio> -->
      <my-radio v-model='picked' value='1' label='one: '></my-radio>
      <my-radio v-model='picked' value='2' label='two: '></my-radio>
      {{ `select is：${picked}` }}
    </div>
  </body>
  <Script>
    Vue.component('MyRadio', {
      model: {
        prop: 'modelVal',
        event: 'change',
      },
      // 注意，定义了model的prop，仍然需要在props属性中声明该prop
      props: ['value', 'label', 'modelVal'],
      template: `
        <label style='display: block'>{{ label }}
          <input type="radio" :value='value' :checked='value==modelVal' @change="$emit('change', value)">
        </label>
      `
    })

    new Vue({
      el: '#app',
      data: {
        picked: ''
      }
    })
  </Script>
```

##### 自定义复选框

复选框比较复杂，因为要根据绑定的值的类型进行判断，源码如下：
```html
<div id="app">
  <!-- 这里的v-model就相当于<my-radio :model-val="picked" @change="val => picked = val"></my-radio> -->
  <h4>单独复选框：</h4>
  <my-checkbox v-model='picked1' value='1' label='one: '></my-checkbox>
  <my-checkbox v-model='picked2' value='2' label='two: ' true-value="yes" false-value="no"></my-checkbox>
  {{ `picked1 is: ${picked1}, picked2 is: ${picked2}` }}
  <h4>多选复选框：</h4>
  <my-checkbox v-model='picked' value='3' label='three: '></my-checkbox>
  <my-checkbox v-model='picked' value='4' label='four: '></my-checkbox>
  {{ `picked is：[${picked}]` }}
</div>
<script>
Vue.component('MyCheckbox', {
  model: {
    prop: 'modelVal',
    event: 'change',
  },
  props: {
    value: {
      type: String
    },
    modelVal: {
      default: false
    },
    label: {
      type: String,
      required: true
    },
    trueValue: {
      default: true
    },
    falseValue: {
      default: false
    }
  },
  template: `
    <label style='display: block'>{{ label }}
      <input type="checkbox" :value='value' :checked='shouldBeChecked' @change='updateInput'>
    </label>
  `,
  computed: {
    shouldBeChecked() {
      // 如果modelVal是数组，则根据modelVal是否包含该checkbox的value来判断是否选中
      if (Array.isArray(this.modelVal)) return this.modelVal.includes(this.value)
      // 否则，就根据modelVal和trueValue是否相等来判断是否选中
      return this.modelVal === this.trueValue
    }
  },
  methods: {
    updateInput(event) {
      // 首先判断原生的checkbox是否选中
      const isChecked = event.target.checked
      if (Array.isArray(this.modelVal)) {
        // 如果modelVal是数组，则复制该数组
        let newVal = [...this.modelVal]
        if (isChecked) {
          // 如果原生的checkbox是选中的，则说明要将这个checkbox的value值加入到数组中
          newVal.push(this.value)
        } else {
          // 否则要从数组中删除这个checkbox的value值
          newVal.splice(newVal.indexOf(this.value), 1)
        }
        // 向外部发射change事件，传出的是新的newVal数组，在外部v-model实际上执行了picked=newVal
        this.$emit('change', newVal)
      } else {
        this.$emit('change', isChecked ? this.trueValue : this.falseValue)
      }
    }
  }
})

new Vue({
  el: '#app',
  data: {
    picked: [],
    picked1: '',
    picked2: ''
  }
})
</script>
```

#### 组件name属性的作用

- [vue组件name的作用小结](https://www.cnblogs.com/likecn/p/12205987.html)

#### 在组件内部引用props

官网有这样一句话：
> Prop 是你可以在组件上注册的一些自定义 attribute。当一个值传递给一个 prop attribute 的时候，它就变成了那个组件实例的一个 property。

举个例子：
```html
<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		<title></title>
		<script src="https://cdn.jsdelivr.net/npm/vue@^2.0.0/dist/vue.min.js"></script>
	</head>
	<body>
		<div id="app">
			<tomorrow v-bind:today="today"></tomorrow>
		</div>
	</body>
</html>
<script>
	Vue.component('tomorrow', {
		props: ['today'],
		template: '<h2>Tomorrow is {{tomorrow}}</h2>',
		computed: {
			tomorrow: function() {
				this.today.setDate(this.today.getDate() + 1)
				return this.today.toLocaleDateString()
			}
		}
	})

	var app = new Vue({
		el: '#app',
		data: {
			today: new Date()
		}
	})
</script>
```
注意各种眼花缭乱的传值，特别注意当today作为`prop`传递给组件的时候，它就变成了组件实例的一个属性，在内部引用`today`的值的时，需要使用`this.today`进行引用。

#### prop的大小写

官网关于组件的`prop`大小写一节没太明白，测试结果如下：

在html中调用组件，通过属性传值时，如果有短横线，则将短横线去掉，将短横线后面的单词的第一个字母大写。如果没有短横线，则将所有字母转换成小写，因此下面两种写法都可以：
1. 有短横线
```html
<sayhello guest-name="telecomshy"></sayhello>
```
则作为prop传入组件时，去掉短横线，n变大写：
```javascript
Vue.component('sayhello',  {
    props: ['guestName'], //不能是guestname, guest-name等其它形式
    template: '<h2>hello {{guestname}}</h2>'
})
```
2. 没有短横线
```html
<sayhello guestName="telecomshy"></sayhello>
```
则作为prop传入组件时，所有字母都变小写：
```javascript
Vue.component('sayhello',  {
    props: ['guestname'], //不能是guestName, guest-name等其它形式
    template: '<h2>hello {{guestname}}</h2>'
})
```

#### 如何理解非Prop属性

非prop属性是指调用组件的时候，其属性并未通过`prop`明确的传递给组件，对于这种属性，默认会添加在组件的根节点上--因为组件只允许有一个根节点，例子如下：
```html
<base-input v-model="name" placeholder="Enter your name"></base-input>
```
组件定义如下：
```javascript
Vue.component('base-input', {
    props: ['value'],
    template: `<input v-bind:value="value" v-on:input="$emit('input', $event.target.value)">`
})
```
其中`placeholder`并未通过`props`传递给组件，但是在浏览器中查看最后渲染的结果为:
```html
<input placeholder="Enter your name">
```
说明非prop属性已经添加在组件的根节点。注意，未通过`props`传递，说明属性并未成为组件实例的property。因此在内部无法通过`this.attr`这样的形式引用。但由于已经添加在组件的根节点，因此可以在组件挂载以后，通过`this.$el.getAttribute(attr)`的方式来引用，如下：
```javascript
Vue.component('base-input', {
    props: ["value"],
    template: `<input v-bind:value="value" v-on:input="$emit('input', $event.target.value)">`,
    mounted:function(){
        console.log(this.$el.getAttribute("placeholder"))
    }
})
```
控制台会输出`Enter your name`。注意，不能在`data`里面引用`this.$el`，这与Vue实例的生命周期有关，具体看Vue实例的生命周期一节。

#### 禁用继承属性以及利用`$attrs`指定继承属性的元素

禁用继承指不允许非prop的属性传递，只要在组件上设置`inheritAttrs`为`False`即可。现在如果有这样的需求，比如你希望非prop能进行传递，但是不想把它添加在组件的根节点上，而是希望添加到你指定的元素，则可以利用组件实例的`$attr`属性。它是一个对象，包含所有非`prop`属性（除了`class`和`style`属性），如下：
```html
<base-input label="your name:" v-model="name" placeholder="Enter your name"></base-input>
```
组件定义如下：
```javascript
Vue.component('base-input', {
    inheritAttrs: false,
    props: ['label', 'value'],
    template: `
        <label>
          {{ label }}
          <input
            v-bind="$attrs"
            v-bind:value="value"
            v-on:input="$emit('input', $event.target.value)"
          >
        </label>
      `
})
```
如上，由于`inheritAttrs`为`false`，所以所有未通过`props`传递的属性不会自动添加在根节点`label`上，但是这些属性，除了`class`和`style`会保存在组件实例的`$attrs`中，因此可以手动指定接收这些属性的dom元素，上例直接传递给组件内部的`input`元素。注意，`v-bind="$attrs"`的写法，以及`v-model`在组件上的使用。这样，`base-input`的任意属性可以直接传递给组件内部的`input`元素，操作`base-input`就好像在操作原生的`input`元素一样。

#### 使用`text/x-template`方式定义组件模板

写组件的时候，最麻烦的莫过于写模板代码，各种引号齐飞，难以利用工具进行格式美化。有一个技巧就是在`<script></script>`标签里写模板，只需要把`type`属性设置为`text/x-template`，如下：
```html
<div id="app">
    <my-component></my-component>
    <script type="text/x-template" id="my-component">
        <div>
            <p>This is the content of component</p>
            <p>Hello Vue!</p>
        </div>				
    </script>
</div>
```
然后定义这个组件：
```javascript
let myComponent = {
    template: "#my-component"
};
```
要注意的就是，需要在`x-template`中定义一个id，在组件定义中通过`#id`进行引用。另外就是，牢记所有组件只能有一个根元素。

#### 组件缓存和组件复用

组件缓存和组件复用是不同的，缓存是添加了keep-alive，会把组件缓存起来，不同的路由切换（这里的路由指对应不同的组件）仍然是同一个组件，创建以后不会触发create，虚拟dom没有变化的话也不会触发update，但是每次切换会触发activate。而复用是指有动态参数的路由切换时候，组件会复用。

关于缓存要注意的是：
1. Vue3中，如果页面使用keepalive保存缓存，在页面切换时，会触发activated钩子，不会触发mounted钩子，同时创建组件时也会触发activated，因此如果使用keepalive的组件要获取数据，应该使用activate。
2. 同样Vue3中，activate中的错误无法被全局错误处理捕获，猜测可能是因为整个生命周期是一个promise chain链，全局错误处理只能捕获这个chain链中未捕获的错误，而activated是在组件已经创建以后触发，不在这个chain链中。

### Vue3组合式API

#### ref和reactive

- [VUE 3 COMPOSITION API: REF VS REACTIVE](https://www.danvega.dev/blog/2020/02/12/vue3-ref-vs-reactive/)
- [ref vs reactive in Vue 3](https://stackoverflow.com/questions/61452458/ref-vs-reactive-in-vue-3)

#### 注意事项

1. 组合式api中，通过计算属性返回的state的值，要用`.value`访问。

## Vue Router

### 路由基本配置

特别注意：如果不是html5的history的模式，在浏览器里面写路由，或者在代码里面配置路由都是从`/#/`后面开始，一开始不知道，走很多弯路。

### 嵌套路由

嵌套路由要注意，预期说是嵌套路由，不如说是嵌套组件，首先子路由的`path`不加`/`是相对路由，接着父路由，如果加了`/`，就是绝对路由，是从根目录算起，如以下的路由配置：
```javascript
path: "/index",
component: () => import("layouts/MainLayout.vue"),
children: [
  {
    path: "/main",
    component: () => import("pages/Index.vue"),
  }
]
```
如果子路由是"/main"，则访问的时候的地址是`http://localhost:8080/#/main`。如果子路由的`path`是"main"，则访问地址为`http://localhost:8080/#/index/main`。但是不管那种`path`配置，浏览器都会先渲染父路由的组件，然后再渲染子路由的组件，因此都可以正常访问。

### 传参的几种方式

1. 使用`name`传值
在`route`配置中，指定了`route`的`name`以后，可以在组件中使用：
```html
<p>我是router-name: {{$route.name}}</p>
```
2. 使用`to`参数来传值
一般情况下，to参数是一个字符串，表示下一跳的路由，但是它也可以是对象，如：
```html
<router-link v-bind:to="{name:'xxx',params:{key:value}}"></router-link>
```
这里`name`要和定义路由时候配置的一样，使用的时候，语法如下：
```html
<p>传递的名字是：{{$route.params.username}}</p>
```
3. 使用url来传参
定义`route`的时候加冒号表示是变量，如下：
```javascript
{path: 'myjob/:jobid/:jobtitle', name='myjob', component:myjob}
```
在引用组件里面照常定义路由链接：
```html
<router link to="/myjob/12/web engineer">我的工作</router-link>
```
在组件里面，就可以通过绑定的变量来获取值：
```html
<a>jobid是:{{$route.params.jobid}}<a>
<a>jobtitle是:{{$route.params.jobtitle}}</a>
```
注意，url传参和`params`传参不能混用，也就是说有了`path`，就不能有`params`参数。

### 单页面多路由

一条路由不光可以对应一个组件，还可以注入多个组件，只要在定义路由的时候`component`参数是个对象，指定每个组件的名称，使用`<route view>`的时候标注注入的是哪个组件就行了。唯一要注意的是，注入时候没有指定名称参数的组件，引用的时候名称为`default`，如下：
```javascript
// 定义路由
components: {
  default:HelloWord,
  left:leftvue,
  right:rightvue
}
// 注入
<router-view/>
<router-view name="left"/>
<router-view name="right"/>
```

### 路由组件传参

普通的通过路由传参，需要在调用的组件里面通过`<div>User {{ $route.params.id }}</div>`的方式调用传递进来的参数，但是这样，如果是父组件调用这个组件，此时就会报错，组件就和路由绑定了。可以在编写路由的时候，通过`props`参数进行解绑。有3种方式：
1. 布尔模式
当`props=true`时，在组件里面就完全通过`props`进行调用，不用管传进来的参数是路由传递的还是父组件传递进来的。不过要注意的是，在组件里面，仍然可以使用`$route.params`进行调用的传递路由的动态参数。
2. 对象模式
这个情况下，只不过是给组件传递一个静态`props`，路由的动态参数在组件里面仍然只能使用`$route.params`的方式来调用。
3. 函数模式
此时函数是`(route)=>{}`的模式，`route`对象是自动传递到函数的参数。

要理解的是整个实现的过程，只要路由里面有动态参数，则会将`route.params`设置为组件的属性，然后自动将动态参数作为`props`参数传递给组件。

### 导航守卫

#### 解析过程

所谓导航守卫就是指路由发生变化的时候，自动触发的一些函数而已。要注意，可能在3个不同的地方触发函数，分别称为全局守卫、路由守卫以及组件内的守卫，这3个守卫分别定义在不同的地方。要理解不同的守卫，就要理解路由跳转的执行过程：
1. 导航被触发。
2. 在失活的组件里调用离开`beforeRouteLeave`守卫。<font color='red'>**全局**</font>
3. 调用全局的`beforeEach`守卫。<font color='red'>**全局**</font>
4. 在重用的组件里调用`beforeRouteUpdate`守卫 (2.2+)。<font color='red'>**组件，特别注意，查询参数的变化，即类似`get`请求的`/user?id=5`这种类型的路由会重用组件，不会重新渲染，因此不会调用组件内守卫`beforeRouteEnter`，此时会调用`beforeRouteUpdate`。注意，这个守卫函数里，组件实例已经被创建，`this`是可以使用的。**</font>
5. 在路由配置里调用`beforeEnter`<font color='red'>**路由**</font>
6. 解析异步路由组件。
7. 在被激活的组件里调用`beforeRouteEnter`。<font color='red'>**组件，特别注意，在这个函数里不能调用组件实例`this`，因为它在渲染该组件的对应路由被`confirm`前调用，此时组件实例还没有创建。如果要用到组件，可以向`next`函数传递一个函数作为回调函数，函数的第一个参数就是组件实例，因为在第12步的时候，会调用这个回调函数，那时候组件就已经创建好了。**</font>
8. 调用全局的 beforeResolve 守卫 (2.5+)。<font color='red'>**全局,它和`router.beforeEach`的区别是在导航被确认之前，同时在所有组件内守卫和异步路由组件被解析之后，解析守卫就被调用。**</font>
9. 导航被确认。
10. 调用全局的 afterEach 钩子。<font color='red'>**全局，注意它不会接受`next`函数也不会改变导航本身。**</font>
11. 触发 DOM 更新。
12. 用创建好的实例调用`beforeRouteEnter`守卫中传给`next`的回调函数。

测试发现：
1. 如果组件没有重新渲染，比如动态参数，组件复用，比如增加或者改变查询参数，在浏览器里回车都不会触发`beforeRouteEnter`，但是如果点击刷新按钮，则都会触发`beforeRouteEnter`。而`beforeRouteUpdate`是只要路由发生了变化，则会触发一次，之后再刷新或者回车都不会触发。
2. 所有的守卫函数中，一定要调用`next`方法，否则不会`resolved`这个钩子（这里不太明白官网的`resolved`是什么意思，感觉应该是指解析下一个守卫函数）。测试的效果来看，如果不加`next`，就是执行完某个守卫函数以后就不再往下继续执行了，表现出来组件不会被渲染，都是空白，只有所有的守卫函数（个人理解就是官网的钩子）全部执行完，导航的状态才会变成`confirmed`（确认）。

#### `beforeRouteUpdate`和`beforeUpdate`的区别

beforeUpdate是数据更改操作导致虚拟 DOM 重新渲染时被调用，而beforeRouteUpdate仅仅只在组件被复用，即一个带有动态参数的路径 `/users/:id`，在 `/users/1` 和 `/users/2` 之间跳转的时候，这种情况下会被调用。从别的路由跳转到当前路由，比如`/users`跳转到`/users/:id`，会触发beforeUpdate，但是不会触发beforeRouteUpdate。具体来说，带动态参数的路径更新的时候，beforeUpdate和beforeRouteUpdate都会触发。流程如下：
- 路由变化，触发beforeRouteUpdate
- 路由更新以后，把动态参数传递给组件，从而又触发beforeUpdate

所以使用routeUpdate一定要特别小心，在别的钩子函数中如果某些数据更改操作导致虚拟 DOM 重新渲染，都会触发routeUpdate，如果在beforeRouteUpdate或者create钩子函数中，更新了el-form绑定的model数据，会意外的触发beforeUpdate。

### 导航故障

#### 检测重定向

`router.currentRoute`总是指当前的路由。比如使用`await router.push()`以后,`router.currentRoute`指的是跳转后的页面。其`redirectedFrom`指重定向的来源，是一个解析出的路由地址，普通的路由的话，其值为`undefined`。

### 注意事项

1. router.push并不是直接就返回一个Promise，比如push的路由错误的话，这个过程是同步的，会在当前作用域下抛出错误，并不在promise链上。
2. Vue3中，组件的`beforeRouteUpdate`钩子函数，只有在与其绑定的路由发生变化时，才会触发。路由变化时，组件包含的子组件的`beforeRouteUpdate`钩子函数不会触发，但是奇怪的是，如果子组件的`beforeRouteUpdate`钩子函数是在`setup`中，就会触发。

## Vuex

### `state`,`mutation`,`getter`和`action`

#### `state`

`state`相当于属性，定义的时候直接定义一个属性即可：
```javascript
state: {
    count: 0
}
```

在组件里面调用的时候把它放在计算属性里面，直接返回，如下：
```javascript
computed: {
    count () {
      return this.$store.state.count
    }
}
```

另外可以使用`mapState`,`mapState`是一个辅助函数，让开发人员少写一些代码，它接受对象或者数组，返回一个对象。先看传入一个对象的情况，相当于`mapState`自动把`this.$store.state`转换成`state`，比如上面的例子，使用`mapState`可以这样写：

```javascript
computed: mapState({
    count(state){
        return state.count
    }
})
```
或者这样：
```javascript
computed: mapState({
    count: state=>state.count
})
```
甚至可以这样：
```javascript
computed: mapState({
    count: 'count'  // 字符串'count'相当于state=>state.count
})
```
如果本地属性和state属性名称一致，还有更简单的写法：
```javascript
computed: {
    ...mapState(['count'])  // 传入名称字符串的数组
}
```
注意，如果要获得局部状态，则必须使用常规函数：
```javascript
mapState({
    count(state){
        return state.count + this.localcount
    }
})
```

另外还要注意的是，如果是模块结构的写法，比如`moduleA`模块的`state`：
```javascript
mapState({
    count: state => state.moduleA.count
})
```

#### `mutation`

`mutation`相当于一个方法，它接受一个`state`和一个`payload`作为参数，主要对`state`进行操作，没有返回值，`payload`是可选的格外参数，名字可以随便取，注意，只能多传一个`payload`，如果想传多个参数的话打包成对象作为`payload`传入：
```javascript
mutations: {
    addcount(state，val) {
        state.count = state.count + val
    }
}
```
在组件里面调用的时候放在`method`里面，通过`store`的`commit`方法来调用`mutations`：
```javascript
method:{
    addcount(val){
        this.$store.commit('addcount', val)
    }
}
```
如果传递的不是一个值，而是一个对象，比如这样的定义：
```javascript
mutations: {
    addcount(state，payload) {
        state.count = state.count + payload.count
    }
}
```
调用的时候有两种语法：
```javascript
method:{
    addcount(val){
        this.$store.commit('addcount', {count: val})
    }
}
```
或者传给`commit`一个对象：
```javascript
method:{
    addcount(val){
        this.$store.commit({
            type: 'addcount',
            count: val
        })
    }
}
```

#### `getter`

`getter`可以理解为`store`的计算属性，也是一个方法，接受参数为`state`,`getters`，如果是模块形式，还可以接收2个参数为`rootState`和`rootGetters`，从而可以访问根节点的`state`和`getter`。注意，既然是计算属性，`getter`是有返回值的。
```javascript
getters:{
    newcount(state, getter, rootState, rootGetters){
        return state.count + rootState.count
    }
}
```
既然是计算属性，那么在组件里面调用的时候也是放在`computed`里面：
```javascript
computed: {
    newcount(){
        return this.$store.getters.newcount
    }
}
```
注意，如果是模块形式,比如这个`getter`是模块`moduleA`的`getter`，调用的方式是：
```javascript
return this.$store.getters["moduleA/newcount"]
```
注意，模块形式的`mapGetter`辅助函数的写法为：
```javascript
mapGetter({
    newcount: "moduleA/newcount"
})
```
和`state`不同，`getter`的`mapGetter`函数不需要传入任何参数。

#### `action`

`action`和`mutation`类似，也是一个方法，不过它是异步的，2点要注意：
1. `action`不能直接操作`state`，只能调用（也就是在`action`里面`commit`）`mutation`，主要是实现异步功能。
2. 它接收的第一个参数和`mutation`不一样，`mutation`第一个参数是`state`，而它是一个`context`，`context`类似一个扩展的`store`，第二个参数和`mutation`一样，也是一个`payload`。
```javascript
actions: {
  incrementAsync ({ commit }) {  //{ commit }是ES6的写法，直接提取`context`的`commit`方法
    setTimeout(() => {
      commit('increment')
    }, 1000)
  }
}
```

`action`通过`dispatch`触发，返回一个`promise`。因此可以这样使用：
```javascript
store.dispatch('actionA').then(() => {
  // ...
})
```
注意，Vue3中，dispatch返回的是一个promise,但是很奇怪的是，如果直接在action里面return，可以被dispatch的then捕获，但是如果action里面直接抛出错误，却不会被dispatch的catch捕获，但是会被全局捕获。而如果包装在一个promise里面抛出错误，则会被dispatch的catch捕获。类似下面封装表现出来的效果：
```javascript
try{
   result = action(context, payload)
   return new Promise((resolve)=>{
      resolve(result)
   })
}catch(err){
  throw err
}
```
反正记住一点，action里面抛出的错误不会被dispatch的catch捕获。

### 模块化

要注意，模块化以后，如果不用命名空间，只有`state`需要加模块名字，`getter`，`mutation`和`action`全部注册在全局命名空间，和之前一样的访问。

### 命名空间

默认情况下，模块内部的`action`、`mutation`和`getter`是注册在全局命名空间，如果设置`namespaced`属性为`true`，则在组件里面调用的时候需要指明路径。但是注意，`state`和前面几个都不一样，`state`有没有命名空间都一样，不存在路径一样的调用方式。

简单来说如下面的例子：
```javascript
//namespaced为false
import { mapMutations } from "vuex";

methods: {
    ...mapMutations({
      changecount: "moduleExample/otherMutation"
    })
}

//namespaced为true
import { mapMutations } from "vuex";

methods: {
    ...mapMutations({
      changecount: "otherMutation"
    })
}
```
而在组件里面调用`state`，只能象下面这样，不能`"moduleExample/subusername"`这样的方式调用：
```javascript
computed: mapState({
    username: "username", //username是根节点的state,"username"相当于state=>state.username
    subusername: state => state.moduleExample.subusername,
    count: state => state.moduleExample.count
})
```

## Vite

### vite导入组件时总是提示路由解析错误？

vite和webpack不同，vite路由末尾的`vue`后缀不能省略。

## Vue三方组件

- [vue实现随机验证码功能](https://www.cnblogs.com/web-aqin/p/10796326.html)
- [vue中封装svg-icon组件并使用](https://www.cnblogs.com/lhjfly/p/10756650.html)
- [vue项目使用svg图片](https://blog.csdn.net/txfyteen/article/details/84865157?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control&dist_request_id=67d73161-a5af-4429-9fcf-422113df2b3e&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control)

## 问题收集

## 经典代码收集

主要将官方的例子收集到一处方便查询：
- [动态组件官方基本例子](https://jsfiddle.net/2gadm0xt/1/)
- [动态组件keep alive官方例子](https://jsfiddle.net/chrisvfritz/Lp20op9o/)

## 相关教程

官方教程和重要资料：
- [Vue官网教程](https://cn.vuejs.org/v2/guide/)
- [所有事件名称查询](https://developer.mozilla.org/zh-CN/docs/Web/Events)
- [axios中文文档](https://www.kancloud.cn/yunye/axios/234845)

完整教程：
- [Vue框架Element UI+Mint UI教程汇总](https://www.jianshu.com/p/6295ee6974c9)
- [在pycharm中开发Vue快速入门](https://www.cnblogs.com/lizeqian1994/p/10690157.html)
- [Vue实战开发](https://blog.csdn.net/kevinfan2011/category_8319577.html)
- [Vue路由简明教程](https://my.oschina.net/u/3802541?tab=newest&catalogId=5761331)

环境搭建：
- [windows环境搭建Vue开发环境](https://www.cnblogs.com/zhaomeizi/p/8483597.html)  
- [pycharm新建vue项目文件](https://www.jianshu.com/p/75258845c492)
- [Vue-CLI3使用mock](https://www.jianshu.com/p/37fef45c0381)
- [webpack@4x打包html里面img后src为“[object Module]”问题？](https://segmentfault.com/a/1190000021360248)
- [webpack@4x集成vueload@15x报`vue-loader was used without the corresponding plugin. Make sure to include VueLoaderPlugin in your webpack config.`错误](https://blog.csdn.net/cominglately/article/details/80555210)
- [Dom模版和字符串模版](https://www.cnblogs.com/kunmomo/p/12582436.html)

知识点：
- [Vue生命周期总结](https://segmentfault.com/a/1190000008010666)
- [详解Vue实例生命周期](https://segmentfault.com/a/1190000011381906)
- [Vue登录拦截 登录后继续跳转指定页面](https://blog.csdn.net/qq_34576876/article/details/94356585)
- [Vue登录注册，并保持登录状态](https://segmentfault.com/a/1190000016040068)
- [在vue中使用session Storage和vuex保存用户登录状态](https://www.jianshu.com/p/48c912177167)
- [vuex存储和本地存储(localstorage、sessionstorage)的区别](https://www.cnblogs.com/jsanntq/p/9288144.html)
- [Vue.js中 watch 的高级用法](https://juejin.im/post/5ae91fa76fb9a07aa7677543)
- [谈谈axios配置请求头content-type](https://www.cnblogs.com/dreamcc/p/10752604.html)
- [vue.js - 奇怪的 event 对象](https://www.jianshu.com/p/b078cfe97c92)
- [Vue导入本地Js文件](https://www.jianshu.com/p/9e3e76ccf6e1)
- [Vue 中如何引入第三方 JS 库](https://blog.csdn.net/csdn_yudong/article/details/78795743)
- [关于Vue.use()详解](https://www.jianshu.com/p/89a05706917a)
- [浅谈vue中慎用style的scoped属性](https://www.jb51.net/article/129228.htm)
- [VUE 环境变量 process process.env](https://blog.csdn.net/zcy_csdn123/article/details/100736846)
- [webpack中require.context的作用](https://zhuanlan.zhihu.com/p/59564277)
- [使用require.context实现前端自动化](https://www.cnblogs.com/cangqinglang/p/12671008.html)
- [webpack引用模块路径方式及其原理](http://www.myjscode.com/page/article76.html)
- [vue-cil和webpack中本地静态图片的路径问题解决方案](https://www.cnblogs.com/xiaojingyuan/p/7080768.html)

开发实战：
- [Authentication best practices for Vue](https://blog.sqreen.com/authentication-best-practices-vue/)
- [Vue.js JWT Authentication with Vuex and Vue Router](https://bezkoder.com/jwt-vue-vuex-authentication/)
- [Django+Vue，前后端分离，实现用户权限认证](https://www.jianshu.com/p/902b18a6bd78)

## 引申阅读

- [一口气了解 babel](https://zhuanlan.zhihu.com/p/43249121)